In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from config import DB_USERNAME, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Create a connection string with the postgreSQL info
database_access = f"postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"


In [4]:
# Establish a connection to the PostgreSQL database 
engine = create_engine(database_access)


In [5]:
# Write a SQL query to select data from the existing view in PostgreSQL 
query = 'SELECT * FROM "Wines"'


In [6]:
# Use Pandas to read the data from the SQL query and convert into a DataFrame 
wines_df = pd.read_sql_query(query, engine)


In [7]:
# Display the Dataframe
wines_df.head()

,id,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol_wine,quality,quality_categorisation
0,4799,White,6.2,0.260,0.20,8.00,0.047,35.0,111.0,0.99445,3.11,0.42,10.4,6,1
1,1159,Red,10.2,0.410,0.43,2.20,0.110,11.0,37.0,0.99728,3.16,0.67,10.8,5,0
2,315,Red,7.1,0.350,0.29,2.50,0.096,20.0,53.0,0.99620,3.42,0.65,11.0,6,1
3,31,Red,6.9,0.685,0.00,2.50,0.105,22.0,37.0,0.99660,3.46,0.57,10.6,6,1
4,3010,White,6.7,0.250,0.31,1.35,0.061,30.5,218.0,0.99388,3.16,0.53,9.5,5,0


In [8]:
# Clean the dataframe, removing columns that will not be used in the ML model
cleaned_wines_df = wines_df.drop(columns=['id','type', 'quality'])

# Display the clean dataframe
cleaned_wines_df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol_wine,quality_categorisation
0,6.2,0.260,0.20,8.00,0.047,35.0,111.0,0.99445,3.11,0.42,10.4,1
1,10.2,0.410,0.43,2.20,0.110,11.0,37.0,0.99728,3.16,0.67,10.8,0
2,7.1,0.350,0.29,2.50,0.096,20.0,53.0,0.99620,3.42,0.65,11.0,1
3,6.9,0.685,0.00,2.50,0.105,22.0,37.0,0.99660,3.46,0.57,10.6,1
4,6.7,0.250,0.31,1.35,0.061,30.5,218.0,0.99388,3.16,0.53,9.5,0


In [9]:
# Split our preprocessed data into our features and target arrays
X = cleaned_wines_df.drop("quality_categorisation", axis=1)
y = cleaned_wines_df["quality_categorisation"]


In [10]:

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Convert labels to float
y_train = y_train.astype(float)
y_test = y_test.astype(float)

# Create a StandardScaler instance and fit it to the training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



## Model Optimisation Attempts

### Attempt 1. Initial Model

In [11]:
#Define deep neural network model
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=12, input_dim=X_train_scaled.shape[1], activation="relu"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Third hidden layer
model.add(tf.keras.layers.Dense(units=4, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


2024-06-06 20:55:26.855792: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 1s 2ms/step - loss: 0.8193 - accuracy: 0.5868
Epoch 2/100
143/143 [==============================] - 0s 1ms/step - loss: 0.6446 - accuracy: 0.6426
Epoch 3/100
143/143 [==============================] - 0s 1ms/step - loss: 0.6361 - accuracy: 0.6466
Epoch 4/100
143/143 [==============================] - 0s 1ms/step - loss: 0.6343 - accuracy: 0.6428
Epoch 5/100
143/143 [==============================] - 0s 2ms/step - loss: 0.6313 - accuracy: 0.6437
Epoch 6/100
143/143 [==============================] - 0s 3ms/step - loss: 0.6285 - accuracy: 0.6475
Epoch 7/100
143/143 [==============================] - 0s 2ms/step - loss: 0.6253 - accuracy: 0.6488
Epoch 8/100
143/143 [==============================] - 0s 2ms/step - loss: 0.6213 - accuracy: 0.6457
Epoch 9/100
143/143 [==============================] - 0s 2ms/step - loss: 0.6186 - accuracy: 0.6525
Epoch 10/100
143/143 [==============================] - 0s 2ms/step - loss: 0.6240 - accura

In [13]:
# Evaluate the performance of model using the loss and predictive accuracy of the model on the test dataset
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 [==============================] - 0s 1ms/step - loss: 0.5363 - accuracy: 0.7082
Loss: 0.5362550616264343, Accuracy: 0.7082051038742065


### Attempt 2. Model with One Less Hidden Layer

In [14]:
#Define deep neural network model removed third layer
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=12, input_dim=X_train_scaled.shape[1], activation="relu"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 1s 2ms/step - loss: 0.6684 - accuracy: 0.6318
Epoch 2/100
143/143 [==============================] - 0s 2ms/step - loss: 0.6434 - accuracy: 0.6387
Epoch 3/100
143/143 [==============================] - 0s 2ms/step - loss: 0.6389 - accuracy: 0.6415
Epoch 4/100
143/143 [==============================] - 0s 3ms/step - loss: 0.6375 - accuracy: 0.6435
Epoch 5/100
143/143 [==============================] - 0s 1ms/step - loss: 0.6340 - accuracy: 0.6475
Epoch 6/100
143/143 [==============================] - 0s 1ms/step - loss: 0.6319 - accuracy: 0.6417
Epoch 7/100
143/143 [==============================] - 0s 1ms/step - loss: 0.6284 - accuracy: 0.6422
Epoch 8/100
143/143 [==============================] - 0s 1ms/step - loss: 0.6265 - accuracy: 0.6413
Epoch 9/100
143/143 [==============================] - 0s 1ms/step - loss: 0.6190 - accuracy: 0.6459
Epoch 10/100
143/143 [==============================] - 0s 2ms/step - loss: 0.6161 - accura

In [16]:
# Evaluate the performance of model using the loss and predictive accuracy of the model on the test dataset
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


61/61 [==============================] - 0s 1ms/step - loss: 0.5307 - accuracy: 0.7231
Loss: 0.5306887626647949, Accuracy: 0.7230769395828247


### Attempt 3. Model with Increased Units and Tanh Activation

In [22]:
#Define deep neural network model increased unit amounts, changed to tanh from relu
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=300, input_dim=X_train_scaled.shape[1], activation="tanh"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=200, activation="tanh"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [23]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 1s 2ms/step - loss: 0.6501 - accuracy: 0.6307
Epoch 2/100
143/143 [==============================] - 0s 3ms/step - loss: 0.6251 - accuracy: 0.6439
Epoch 3/100
143/143 [==============================] - 0s 2ms/step - loss: 0.5982 - accuracy: 0.6657
Epoch 4/100
143/143 [==============================] - 0s 2ms/step - loss: 0.5838 - accuracy: 0.6782
Epoch 5/100
143/143 [==============================] - 0s 2ms/step - loss: 0.5899 - accuracy: 0.6800
Epoch 6/100
143/143 [==============================] - 0s 2ms/step - loss: 0.5626 - accuracy: 0.6908
Epoch 7/100
143/143 [==============================] - 0s 2ms/step - loss: 0.5595 - accuracy: 0.7002
Epoch 8/100
143/143 [==============================] - 0s 3ms/step - loss: 0.5482 - accuracy: 0.7040
Epoch 9/100
143/143 [==============================] - 0s 2ms/step - loss: 0.5397 - accuracy: 0.7095
Epoch 10/100
143/143 [==============================] - 1s 4ms/step - loss: 0.5369 - accura

In [24]:
# Evaluate the performance of model using the loss and predictive accuracy of the model on the test dataset
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


61/61 [==============================] - 0s 2ms/step - loss: 0.5194 - accuracy: 0.7149
Loss: 0.5194076895713806, Accuracy: 0.7148718237876892


### Attempt 4. Model with Increased Units, Tanh Activation, and L2 Regularization

In [25]:
#Define deep neural network model increased unit amounts, changed to tanh from relu
model = tf.keras.models.Sequential()

# First hidden layer with L2 regularization
model.add(Dense(units=300, input_dim=X_train_scaled.shape[1], activation="tanh", kernel_regularizer=l2(0.001)))

# Second hidden layer with L2 regularization
model.add(Dense(units=200, activation="tanh", kernel_regularizer=l2(0.001)))

# Output layer
model.add(Dense(units=1, activation="sigmoid", kernel_regularizer=l2(0.001)))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 1s 2ms/step - loss: 0.9020 - accuracy: 0.6248
Epoch 2/100
143/143 [==============================] - 0s 3ms/step - loss: 0.7977 - accuracy: 0.6470
Epoch 3/100
143/143 [==============================] - 0s 3ms/step - loss: 0.7615 - accuracy: 0.6618
Epoch 4/100
143/143 [==============================] - 0s 2ms/step - loss: 0.7140 - accuracy: 0.6677
Epoch 5/100
143/143 [==============================] - 0s 2ms/step - loss: 0.6792 - accuracy: 0.6756
Epoch 6/100
143/143 [==============================] - 0s 2ms/step - loss: 0.6602 - accuracy: 0.6793
Epoch 7/100
143/143 [==============================] - 0s 2ms/step - loss: 0.6563 - accuracy: 0.6844
Epoch 8/100
143/143 [==============================] - 0s 2ms/step - loss: 0.6273 - accuracy: 0.6967
Epoch 9/100
143/143 [==============================] - 0s 3ms/step - loss: 0.6215 - accuracy: 0.6967
Epoch 10/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6122 - accura

In [27]:
# Evaluate the performance of model using the loss and predictive accuracy of the model on the test dataset
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


61/61 [==============================] - 0s 2ms/step - loss: 0.6326 - accuracy: 0.6764
Loss: 0.632624626159668, Accuracy: 0.6764102578163147


### Attempt 5. Model with Dropout and L2 Regularization

In [28]:
# Define deep neural network model
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=120, activation="relu", input_dim=X_train_scaled.shape[1], kernel_regularizer=l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=80, activation="relu", kernel_regularizer=l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))

# Third hidden layer
model.add(tf.keras.layers.Dense(units=40, activation="relu", kernel_regularizer=l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [29]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 1s 2ms/step - loss: 1.6258 - accuracy: 0.5467
Epoch 2/100
143/143 [==============================] - 0s 2ms/step - loss: 0.8904 - accuracy: 0.5601
Epoch 3/100
143/143 [==============================] - 0s 2ms/step - loss: 0.7991 - accuracy: 0.6061
Epoch 4/100
143/143 [==============================] - 0s 2ms/step - loss: 0.7719 - accuracy: 0.6323
Epoch 5/100
143/143 [==============================] - 0s 2ms/step - loss: 0.7681 - accuracy: 0.6303
Epoch 6/100
143/143 [==============================] - 0s 2ms/step - loss: 0.7595 - accuracy: 0.6329
Epoch 7/100
143/143 [==============================] - 0s 3ms/step - loss: 0.7490 - accuracy: 0.6362
Epoch 8/100
143/143 [==============================] - 0s 3ms/step - loss: 0.7443 - accuracy: 0.6365
Epoch 9/100
143/143 [==============================] - 0s 2ms/step - loss: 0.7376 - accuracy: 0.6378
Epoch 10/100
143/143 [==============================] - 0s 2ms/step - loss: 0.7344 - accura

In [30]:
# Evaluate the performance of model using the loss and predictive accuracy of the model on the test dataset
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 [==============================] - 0s 1ms/step - loss: 0.5518 - accuracy: 0.7185
Loss: 0.551754891872406, Accuracy: 0.7184615135192871


### Attempt 6. Model with One Less Hidden Layer and Batch Size

In [31]:
#Define deep neural network model removed third layer
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=12, input_dim=X_train_scaled.shape[1], activation="relu"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Third hidden layer
# model.add(tf.keras.layers.Dense(units=4, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [32]:
#Train the model
model.fit(X_train, y_train, epochs=100, batch_size=10)

Epoch 1/100
455/455 [==============================] - 1s 2ms/step - loss: 0.8478 - accuracy: 0.6338
Epoch 2/100
455/455 [==============================] - 1s 1ms/step - loss: 0.6383 - accuracy: 0.6365
Epoch 3/100
455/455 [==============================] - 1s 1ms/step - loss: 0.6322 - accuracy: 0.6371
Epoch 4/100
455/455 [==============================] - 1s 1ms/step - loss: 0.6204 - accuracy: 0.6450
Epoch 5/100
455/455 [==============================] - 1s 2ms/step - loss: 0.6127 - accuracy: 0.6477
Epoch 6/100
455/455 [==============================] - 1s 2ms/step - loss: 0.5953 - accuracy: 0.6620
Epoch 7/100
455/455 [==============================] - 1s 2ms/step - loss: 0.5825 - accuracy: 0.6701
Epoch 8/100
455/455 [==============================] - 1s 2ms/step - loss: 0.5781 - accuracy: 0.6712
Epoch 9/100
455/455 [==============================] - 1s 1ms/step - loss: 0.5652 - accuracy: 0.6862
Epoch 10/100
455/455 [==============================] - 1s 1ms/step - loss: 0.5663 - accura

In [33]:
# Evaluate the performance of model using the loss and predictive accuracy of the model on the test dataset
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 [==============================] - 0s 1ms/step - loss: 0.5619 - accuracy: 0.7072
Loss: 0.5618614554405212, Accuracy: 0.7071794867515564


### 7. Model with Multiple Dropout Layers and Regularization

In [34]:
# Define the model
model = Sequential()

## Input layer
model.add(Dense(7, activation='tanh', input_shape=(X_train_scaled.shape[1],)))

# Hidden layers with dropout and L2 regularization
model.add(Dense(1, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(6, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(6, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(6, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dropout(0.5))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model with binary_crossentropy loss
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)

In [37]:
# Train the model
history = model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping, reduce_lr])

Epoch 1/10
143/143 [==============================] - 0s 3ms/step - loss: 0.6583 - accuracy: 0.6384 - val_loss: 0.6666 - val_accuracy: 0.6205 - lr: 0.0010
Epoch 2/10
143/143 [==============================] - 0s 2ms/step - loss: 0.6578 - accuracy: 0.6384 - val_loss: 0.6655 - val_accuracy: 0.6205 - lr: 0.0010
Epoch 3/10
143/143 [==============================] - 0s 3ms/step - loss: 0.6563 - accuracy: 0.6384 - val_loss: 0.6648 - val_accuracy: 0.6205 - lr: 0.0010
Epoch 4/10
143/143 [==============================] - 0s 2ms/step - loss: 0.6572 - accuracy: 0.6384 - val_loss: 0.6645 - val_accuracy: 0.6205 - lr: 0.0010
Epoch 5/10
143/143 [==============================] - 0s 2ms/step - loss: 0.6563 - accuracy: 0.6384 - val_loss: 0.6643 - val_accuracy: 0.6205 - lr: 0.0010
Epoch 6/10
143/143 [==============================] - 0s 2ms/step - loss: 0.6553 - accuracy: 0.6384 - val_loss: 0.6643 - val_accuracy: 0.6205 - lr: 0.0010
Epoch 7/10
143/143 [==============================] - 0s 2ms/step - lo

In [38]:
# Evaluate the performance of model using the loss and predictive accuracy of the model on the test dataset
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 [==============================] - 0s 2ms/step - loss: 0.6643 - accuracy: 0.6205
Loss: 0.6642521619796753, Accuracy: 0.620512843132019


## Optimum model
#### The code below demonstrates the model with the best performance

### Using a binning method to preprocess the data

In [39]:
# Define the bin edges for the Free Sulfur Dioxide variable
bins_free_sulfur_dioxide = [0, 21.75, 42.5, 63.25, 84, 289]

# Create labels for the bins
labels = ['low','mid-low','medium','mid-high','high']

In [40]:
# Perform the binning and create a new column with the bin values
cleaned_wines_df['free_sulfur_binned'] = pd.cut(cleaned_wines_df['free_sulfur_dioxide'], bins = bins_free_sulfur_dioxide, labels=labels)
cleaned_wines_df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol_wine,quality_categorisation,free_sulfur_binned
0,6.2,0.260,0.20,8.00,0.047,35.0,111.0,0.99445,3.11,0.42,10.4,1,mid-low
1,10.2,0.410,0.43,2.20,0.110,11.0,37.0,0.99728,3.16,0.67,10.8,0,low
2,7.1,0.350,0.29,2.50,0.096,20.0,53.0,0.99620,3.42,0.65,11.0,1,low
3,6.9,0.685,0.00,2.50,0.105,22.0,37.0,0.99660,3.46,0.57,10.6,1,mid-low
4,6.7,0.250,0.31,1.35,0.061,30.5,218.0,0.99388,3.16,0.53,9.5,0,mid-low


In [41]:
# Display how many records were allocated into each bin
cleaned_wines_df['free_sulfur_binned'].value_counts()

mid-low     2751
low         2230
medium      1275
mid-high     205
high          36
Name: free_sulfur_binned, dtype: int64

In [42]:
# Define the bin edges for the Density feature
bins_density = [0, 0.9903325, 0.993555, 0.9987775, 1, 2]


In [43]:
# Perform the binning and create a new column with the bin values
cleaned_wines_df['density_binned'] = pd.cut(cleaned_wines_df['density'], bins = bins_density, labels=labels)
cleaned_wines_df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol_wine,quality_categorisation,free_sulfur_binned,density_binned
0,6.2,0.260,0.20,8.00,0.047,35.0,111.0,0.99445,3.11,0.42,10.4,1,mid-low,medium
1,10.2,0.410,0.43,2.20,0.110,11.0,37.0,0.99728,3.16,0.67,10.8,0,low,medium
2,7.1,0.350,0.29,2.50,0.096,20.0,53.0,0.99620,3.42,0.65,11.0,1,low,medium
3,6.9,0.685,0.00,2.50,0.105,22.0,37.0,0.99660,3.46,0.57,10.6,1,mid-low,medium
4,6.7,0.250,0.31,1.35,0.061,30.5,218.0,0.99388,3.16,0.53,9.5,0,mid-low,medium


In [44]:
# Display how many records were allocated into each bin
cleaned_wines_df['density_binned'].value_counts()

medium      3594
mid-low     1911
low          496
mid-high     349
high         147
Name: density_binned, dtype: int64

In [45]:
# Define the bin edges for the Residual Sugar feature
bins_sugar = [0, 5.45, 10.3, 15.15, 20, 100]

In [46]:
# Perform the binning and create a new column with the bin values
cleaned_wines_df['sugar_binned'] = pd.cut(cleaned_wines_df['residual_sugar'], bins = bins_sugar, labels=labels)
cleaned_wines_df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol_wine,quality_categorisation,free_sulfur_binned,density_binned,sugar_binned
0,6.2,0.260,0.20,8.00,0.047,35.0,111.0,0.99445,3.11,0.42,10.4,1,mid-low,medium,mid-low
1,10.2,0.410,0.43,2.20,0.110,11.0,37.0,0.99728,3.16,0.67,10.8,0,low,medium,low
2,7.1,0.350,0.29,2.50,0.096,20.0,53.0,0.99620,3.42,0.65,11.0,1,low,medium,low
3,6.9,0.685,0.00,2.50,0.105,22.0,37.0,0.99660,3.46,0.57,10.6,1,mid-low,medium,low
4,6.7,0.250,0.31,1.35,0.061,30.5,218.0,0.99388,3.16,0.53,9.5,0,mid-low,medium,low


In [47]:
# Display how many records were allocated into each bin
cleaned_wines_df['density_binned'].value_counts()

medium      3594
mid-low     1911
low          496
mid-high     349
high         147
Name: density_binned, dtype: int64

In [48]:
# Clean the dataframe, removing columns that will not be used in the ML model
cleaned_wines_df2 = cleaned_wines_df.drop(columns=['free_sulfur_dioxide','density','free_sulfur_binned', 'density_binned', 'residual_sugar','sugar_binned' ])

# Display the clean dataframe
cleaned_wines_df2.head()

,fixed_acidity,volatile_acidity,citric_acid,chlorides,total_sulfur_dioxide,ph,sulphates,alcohol_wine,quality_categorisation
0,6.2,0.260,0.20,0.047,111.0,3.11,0.42,10.4,1
1,10.2,0.410,0.43,0.110,37.0,3.16,0.67,10.8,0
2,7.1,0.350,0.29,0.096,53.0,3.42,0.65,11.0,1
3,6.9,0.685,0.00,0.105,37.0,3.46,0.57,10.6,1
4,6.7,0.250,0.31,0.061,218.0,3.16,0.53,9.5,0


### Encode the non numerical columns using Get Dummies

In [49]:
# Encode the binned columns
df_dummies = pd.get_dummies(cleaned_wines_df[['free_sulfur_binned','density_binned', 'sugar_binned']])
df_dummies.head(10)

,free_sulfur_binned_low,free_sulfur_binned_mid-low,free_sulfur_binned_medium,free_sulfur_binned_mid-high,free_sulfur_binned_high,density_binned_low,density_binned_mid-low,density_binned_medium,density_binned_mid-high,density_binned_high,sugar_binned_low,sugar_binned_mid-low,sugar_binned_medium,sugar_binned_mid-high,sugar_binned_high
0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0
5,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
6,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0
7,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0
8,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
9,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0


In [50]:
# Concatenate the encoded dataframe with the other columns that will be used in the ML model
df_wines_concatenated = pd.concat([cleaned_wines_df2, df_dummies], axis=1)

# Display the dataframe
df_wines_concatenated.head(3)

,fixed_acidity,volatile_acidity,citric_acid,chlorides,total_sulfur_dioxide,ph,sulphates,alcohol_wine,quality_categorisation,free_sulfur_binned_low,...,density_binned_low,density_binned_mid-low,density_binned_medium,density_binned_mid-high,density_binned_high,sugar_binned_low,sugar_binned_mid-low,sugar_binned_medium,sugar_binned_mid-high,sugar_binned_high
0,6.2,0.26,0.20,0.047,111.0,3.11,0.42,10.4,1,0,...,0,0,1,0,0,0,1,0,0,0
1,10.2,0.41,0.43,0.110,37.0,3.16,0.67,10.8,0,1,...,0,0,1,0,0,1,0,0,0,0
2,7.1,0.35,0.29,0.096,53.0,3.42,0.65,11.0,1,1,...,0,0,1,0,0,1,0,0,0,0


### Create the ML Model

In [60]:
# Split our preprocessed data into our features and target arrays
X = df_wines_concatenated.drop("quality_categorisation", axis=1)
y = df_wines_concatenated["quality_categorisation"]


In [61]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Fit the StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [62]:
#Define deep neural network model removed third layer
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=200, input_dim=X_train_scaled.shape[1], activation="relu"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=150, activation="relu"))

# # Third hidden layer
# model.add(tf.keras.layers.Dense(units=100, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [65]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 0s 3ms/step - loss: 0.4887 - accuracy: 0.7581
Epoch 2/100
143/143 [==============================] - 0s 2ms/step - loss: 0.4856 - accuracy: 0.7623
Epoch 3/100
143/143 [==============================] - 0s 2ms/step - loss: 0.4864 - accuracy: 0.7636
Epoch 4/100
143/143 [==============================] - 0s 2ms/step - loss: 0.4826 - accuracy: 0.7664
Epoch 5/100
143/143 [==============================] - 0s 2ms/step - loss: 0.4849 - accuracy: 0.7651
Epoch 6/100
143/143 [==============================] - 0s 2ms/step - loss: 0.4882 - accuracy: 0.7587
Epoch 7/100
143/143 [==============================] - 0s 2ms/step - loss: 0.4878 - accuracy: 0.7550
Epoch 8/100
143/143 [==============================] - 0s 2ms/step - loss: 0.4984 - accuracy: 0.7427
Epoch 9/100
143/143 [==============================] - 0s 2ms/step - loss: 0.4853 - accuracy: 0.7590
Epoch 10/100
143/143 [==============================] - 0s 2ms/step - loss: 0.4826 - accura

In [66]:
# Evaluate the performance of model using the loss and predictive accuracy of the model on the test dataset
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 [==============================] - 0s 2ms/step - loss: 0.5419 - accuracy: 0.7544
Loss: 0.5418981909751892, Accuracy: 0.7543589472770691
